<a href="https://colab.research.google.com/github/TarteTatin1212/contents_search/blob/main/contents_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#モジュールのインストール
!pip install langid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941173 sha256=467b71481f3ab47aed04b5b27798322d710cf5f83c38bbf2269ddc2dc1ef752d
  Stored in directory: /root/.cache/pip/wheels/23/c8/c6/eed80894918490a175677414d40bd7c851413bbe03d4856c3c
Successfully built langid


In [9]:
import langid              # 言語識別のためのライブラリ
import pandas as pd        # データ分析のためのライブラリ（データフレームを操作するため）
import json               # JSON形式のデータを操作するためのモジュール
import pytz               # タイムゾーン情報を扱うためのライブラリ
from datetime import datetime, timedelta # 日付と時間を操作するためのモジュール
from apiclient.discovery import build    # Google APIクライアントを構築するためのモジュール
from google.colab import drive           # Google Colab環境でGoogle Driveを操作するためのモジュール


In [4]:
# Google Driveをマウント
drive.mount('/content/drive')

# ディレクトリをSenior_Projectに変更
%cd '/content/drive/MyDrive/Horita_Semi/Senior_Project'

Mounted at /content/drive
/content/drive/MyDrive/Horita_Semi/Senior_Project


In [10]:
%ls

2023年6月1日課題点羅列.gdoc  youtube_data_japanese.csv
api_key.txt                  youtube_data_japanese.json
contents_search.ipynb        youtube_data_other.csv
face_recognition.ipynb       youtube_data_other.json


In [11]:
def read_api_key(filepath):
    """APIキーをファイルから読み込む"""
    with open(filepath, 'r') as file:  # 指定されたファイルパスを読み取りモードで開く
        return file.readline().strip() # ファイルの最初の行を読み取り、余分な空白や改行を除去して返す


In [12]:
def build_youtube_client(api_key):
    """YouTube APIクライアントを初期化"""
    return build("youtube", "v3", developerKey=api_key)  # Google APIのbuild関数を使用してYouTube APIのクライアントをv3バージョンで初期化し、それを返す


In [13]:
def get_search_period():
    """検索期間を取得する関数"""
    current_time_japan = datetime.now(pytz.timezone("Asia/Tokyo"))  # 現在の日本時間を取得

    # 前日の日付を取得
    previous_day = current_time_japan - timedelta(days=1)

    # 前日の18:00と21:00を取得
    start_time = previous_day.replace(hour=18, minute=0, second=0, microsecond=0)
    end_time = previous_day.replace(hour=21, minute=0, second=0, microsecond=0)

    return start_time.isoformat(), end_time.isoformat()  # 開始時刻と終了時刻をISO 8601形式の文字列として返す


In [14]:
def print_search_period_dates():
    """検索期間の日付を出力する関数"""
    start_time, _ = get_search_period()  # 検索期間の開始時刻を取得

    # ISO形式の文字列から日付と時刻を取得
    start_datetime = datetime.fromisoformat(start_time).astimezone(pytz.timezone("Asia/Tokyo"))

    # 日付をフォーマットして出力
    print(f"Search date: {start_datetime.strftime('%Y-%m-%d')} (Japan Time)")


In [15]:
def fetch_videos(youtube, start_time_iso, end_time_iso):
    """指定された期間にアップロードされた動画を検索する関数"""
    # YouTubeのsearchリソースのlistメソッドを用いて動画検索を実行
    return youtube.search().list(
        q="",                   # 検索クエリ（空文字列で全ての動画を対象とする）
        part="id",              # 必須パラメータ。取得するリソースの部分を指定。ここでは動画のIDのみを取得
        type="video",           # 検索するリソースのタイプを指定。ここでは動画のみを対象とする
        publishedAfter=start_time_iso,  # 指定された時刻以降に公開された動画のみを対象とする
        maxResults=100,         # 一度のリクエストで取得する結果の最大数を指定。最大値は50だが、例として100が示されている（エラーになる可能性あり）
        regionCode="JP",        # 公開地域を指定。ここでは日本で公開された動画のみを対象とする
    ).execute()                # 検索リクエストを実行して結果を取得


In [16]:
def print_fetched_videos(youtube, start_time_iso, end_time_iso):
    """指定された期間にアップロードされた動画の情報を出力する関数"""

    # fetch_videos関数で動画情報を取得
    search_response = fetch_videos(youtube, start_time_iso, end_time_iso)

    # 検索結果の動画情報を一つずつ出力
    for item in search_response.get("items", []):
        video_id = item["id"]["videoId"]
        print(f"Video ID: {video_id}")

    print(f"Total videos fetched: {len(search_response.get('items', []))}")


In [17]:
def classify_and_store_videos(search_response, youtube):
    """動画を日本語とその他のカテゴリに分類する関数"""

    # 日本語の動画を保存するリスト
    data_japanese = []
    # その他の言語の動画を保存するリスト
    data_other = []

    # 検索応答から各動画の情報を取得
    for item in search_response["items"]:
        # 動画のIDを取得
        video_id = item["id"]["videoId"]

        # 動画の詳細情報をYouTube APIから取得
        video_response = youtube.videos().list(part="snippet,statistics", id=video_id).execute()

        # 取得した動画の詳細情報を処理
        for item in video_response["items"]:
            # 動画のタイトルを取得
            title = item["snippet"]["title"]

            # タイトルの言語を判定
            lang, _ = langid.classify(title)

            # チャンネルのタイトルを取得
            channel_title = item["snippet"]["channelTitle"]
            # サムネイルのURLを取得
            thumbnail_url = item["snippet"]["thumbnails"]["default"]["url"]
            # 視聴回数を取得
            view_count = item["statistics"]["viewCount"]

            # 上記で取得した情報を辞書にまとめる
            video_info = {
                "title": title,
                "channelTitle": channel_title,
                "thumbnailUrl": thumbnail_url,
                "viewCount": view_count
            }

            # 言語が日本語であればdata_japaneseに、それ以外であればdata_otherに追加
            if lang == 'ja':
                data_japanese.append(video_info)
            else:
                data_other.append(video_info)

    # 2つのリストを返す
    return data_japanese, data_other


In [18]:
def print_video_data(data_japanese, data_other):
    """辞書に格納された動画データをターミナルに出力する関数"""

    print("\n=== 日本語の動画 ===")
    for video in data_japanese:
        print(f"タイトル: {video['title']}")
        print(f"チャンネル: {video['channelTitle']}")
        print(f"サムネイルURL: {video['thumbnailUrl']}")
        print(f"視聴回数: {video['viewCount']}")
        print("---------------------------")

    print("\n=== その他の言語の動画 ===")
    for video in data_other:
        print(f"タイトル: {video['title']}")
        print(f"チャンネル: {video['channelTitle']}")
        print(f"サムネイルURL: {video['thumbnailUrl']}")
        print(f"視聴回数: {video['viewCount']}")
        print("---------------------------")

In [19]:
def save_data_to_files(data_japanese, data_other):
    """データをJSONおよびCSVファイルに保存する関数"""

    # 日本語の動画データをJSONファイルとして保存
    with open('youtube_data_japanese.json', 'w') as json_file:
        # data_japaneseをJSON形式でファイルに書き出し
        json.dump(data_japanese, json_file)

    # その他の言語の動画データをJSONファイルとして保存
    with open('youtube_data_other.json', 'w') as json_file:
        # data_otherをJSON形式でファイルに書き出し
        json.dump(data_other, json_file)

    # 日本語の動画データをpandasのDataFrameオブジェクトに変換
    df_japanese = pd.DataFrame(data_japanese)
    # DataFrameをCSVファイルとして保存（インデックスは保存しない）
    df_japanese.to_csv('youtube_data_japanese.csv', index=False)

    # その他の言語の動画データをpandasのDataFrameオブジェクトに変換
    df_other = pd.DataFrame(data_other)
    # DataFrameをCSVファイルとして保存（インデックスは保存しない）
    df_other.to_csv('youtube_data_other.csv', index=False)


In [20]:
# 上記の関数を使用して、処理を順番に実行する

# 'api_key.txt' ファイルからAPIキーを読み込む
api_key = read_api_key('/content/drive/MyDrive/Horita_Semi/Senior_Project/api_key.txt')

# 読み込んだAPIキーを使ってYouTube APIクライアントを初期化
youtube = build_youtube_client(api_key)

# 日本時間で18:00から21:00までのISO 8601形式の開始時刻と終了時刻を取得
start_time_iso, end_time_iso = get_search_period()

# 検索日を出力
print_search_period_dates()

# 上で取得した期間内にアップロードされた動画をYouTube APIを使用して検索
search_response = fetch_videos(youtube, start_time_iso, end_time_iso)

# 動画情報を出力
print_fetched_videos(youtube, start_time_iso, end_time_iso)

# 検索結果から動画のタイトルをもとに言語を識別し、日本語とその他に分類
data_japanese, data_other = classify_and_store_videos(search_response, youtube)

# データをターミナルに出力
print_video_data(data_japanese, data_other)

# 分類された動画データをJSONおよびCSVファイルに保存
save_data_to_files(data_japanese, data_other)


Search date: 2023-09-14 (Japan Time)
Video ID: U3DzJ2uYI7I
Video ID: JI-wyXnr5WI
Video ID: fpD9hi6i1ro
Video ID: FlyP12rhIso
Video ID: OXmHHvvYbhU
Video ID: pGITu6dv7qE
Video ID: xTdwF8uh5l8
Video ID: A7AL5Fv156Y
Video ID: NONjvWDMdrw
Video ID: KTYQZCXtyLw
Video ID: nuBMnluJfs0
Video ID: kqVTNqQHun0
Video ID: 51KU3j7sDjw
Video ID: oAXdTHblZno
Video ID: SUtgAPH4OHc
Video ID: oc2mOBaCwik
Video ID: KDUtdcU10YA
Video ID: OzzqF517nnQ
Video ID: i1UP4oKPTJY
Video ID: clNhAEL1eOI
Video ID: JIrows5Tdqs
Video ID: ZBmecMX-dJk
Video ID: KM1SR_DJjhY
Video ID: 4FCIOtUVnrY
Video ID: DGbvgq_9Zhw
Video ID: miej-2RkVLI
Video ID: btee5fMyZEg
Video ID: W3RNCmXrf7I
Video ID: giSHPmXprxw
Video ID: FV3bqvOHRQo
Video ID: eSUAamnMc-0
Video ID: Hft7PLP9Lmg
Video ID: 38msjAwqFxA
Video ID: Volcj_C5xTY
Video ID: OVfTlauuWvc
Video ID: RBT2XL9kano
Video ID: PC6bx4C4BAk
Video ID: GfjH1erOv58
Video ID: p0kt4MnxP-0
Video ID: N5oT7uCyO5s
Video ID: 8vYgemosL6M
Video ID: CPEBkMiV_WY
Video ID: ZoC1mHqC24M
Video ID: aP0hzPv